In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
        

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
'''
The Objective - deeply analyze dataset, by experimenting a verity of ML models and technics
--- Main Tasks ---

1. Cleaning and organizing the data
2. Get initial understanding of relations in the data 
3. Clustering – for categorizing data, set “strong” features
4. Regression – seeking the best model to determine “selling price”  


'''

In [ ]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import DBSCAN
from sklearn.cluster import MeanShift
from sklearn.cluster import Birch
from sklearn.cluster import AffinityPropagation
from sklearn.cluster import MiniBatchKMeans

import warnings
#warnings.filterwarnings("ignore")

In [ ]:
import os
os.getcwd()

In [ ]:
df=pd.read_csv('../input/vehicle-dataset-from-cardekho/Car details v3.csv').drop('torque',axis = 1 ).dropna(how = 'all')
df.drop_duplicates(subset = ['name','year','selling_price', 'km_driven', 'seller_type','mileage'], keep = 'first')
df.sample(4)

In [ ]:
df.owner.unique()
# sum(df.owner.str.contains('First | Second | third', case = False))
df['owner'].replace({'First Owner': 1,'Second Owner':2, 'Third Owner':3,'Fourth & Above Owner':4, 'Test Drive Car':6}, inplace = True)
df.owner.unique()

In [ ]:
import datetime as dt
dt.datetime.today().year
df['car_age'] = dt.datetime.today().year - df.year 
df['car_age']
df.drop('year', axis = 1 ) 

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df['max_power'] = df['max_power'].str.strip('bhp')

df['max_power'] = df['max_power'].str.replace(" ", "")
# 
df['max_power'].sort_values(ascending = False).head()#.astype('float64')
df['max_power'] = pd.to_numeric(df['max_power'],errors='coerce')

In [ ]:
df['engine'] = df['engine'].str.replace("CC", "")
df['engine'] = pd.to_numeric(df['engine'], errors= 'coerce')
df['mileage'] = df['mileage'].str.replace("kmpl", "")
df['mileage'] = pd.to_numeric(df['mileage'], errors= 'coerce')
df['mileage'].sample(5)

In [ ]:
# name to brand
df['brand'] = df.name.str.split(' ').str[0]
df.drop(['year','name'], axis =1, inplace = True)

In [ ]:
df.dropna(inplace = True)
# df.isnull().any()
df.isnull().sum()

In [ ]:
# Shuffling the data
df.sample(frac = 1).reset_index(drop = True)

In [ ]:
df.describe()

In [ ]:
import seaborn as sns
fig, ax = plt.subplots(figsize = (10,5))
sns.heatmap(df.corr(), annot = True )

In [ ]:
plt.boxplot(df[['max_power','car_age', 'engine']])
df[['max_power','car_age', 'engine']].describe()

In [ ]:
fig, ax  = plt.subplots(figsize = (10,5)) 
plt.scatter(df.selling_price, df.max_power)
plt.ylabel('Max Power')
plt.xlabel('Selling Price')

### Categorial data into no.


In [ ]:
df.seller_type.unique()

In [ ]:
from sklearn import preprocessing as pre
df.transmission = pre.LabelEncoder().fit_transform(df.transmission)
df.fuel = pre.LabelEncoder().fit_transform(df.fuel)
df.fuel.unique()

### One-Hot Encoding

In [ ]:
# better nameing
df_temp = pd.get_dummies(df.seller_type)
df_temp.iloc[:,0:2]

In [ ]:
print(df_temp.iloc[:,0:2].shape  , df.shape)
df  = pd.concat([df,df_temp.iloc[:,0:2]], axis = 1)
df.shape

In [ ]:
del df_temp
df.drop(['seller_type'],axis = 1, inplace= True)

In [ ]:
df.columns

In [ ]:
df  = pd.get_dummies(df, columns= ['brand'])
df.shape

### K-mean - Basic

In [ ]:
from sklearn.metrics import silhouette_score
model = KMeans(n_clusters=3,max_iter=1000, algorithm = 'full', tol=0.0001 ).fit(df)
model.labels_

In [ ]:
# The silhouette_score gives the average value for all the samples.
# This gives a perspective into the density and separation of the formed
# clusters
silhouette_avg = silhouette_score(df,model.labels_)
print("For n_clusters =", model.n_clusters,
      "The average silhouette_score is :", silhouette_avg)

### Hyper Tuning - the clustering models

In [ ]:
df_num_only = df.iloc[:,np.r_[0:2,5:9]]
df_full_feature = df
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
df_std = sc.fit_transform(df_full_feature)
df_std_num = sc.fit_transform(df_num_only)
print('std values , all features ')
print(df_std)
print('\n std values , only numeric features ')
print(df_std_num)

#### K-mean

In [ ]:
from sklearn.model_selection import ParameterGrid
best_score = -1
n_Kmeans =list(range(3,11,1))
n_Kmeans

In [ ]:
best_score = -1
best_no_Kmeans = 0
inter_no = 0
for i in n_Kmeans:
    for j in [200, 1000, 100000]:
        model = KMeans(n_clusters=i,max_iter= j , algorithm = 'full', tol=0.00001 ).fit(df_std_num)
        silhouette_avg = silhouette_score(df_std_num,model.labels_)
        print("For n_clusters =", model.n_clusters,'inter_no: ',model.max_iter,"The average silhouette_score is :", silhouette_avg)
        if silhouette_avg > best_score:
            best_score =  silhouette_avg
            best_no_Kmeans = model.n_clusters
            inter_no = model.max_iter

In [ ]:
print("Best n_clusters =", best_no_Kmeans,'Max_iter: ',model.max_iter," | Best silhouette_score is :", best_score)

In [ ]:
model = KMeans(n_clusters =  best_no_Kmeans ,max_iter=1000).fit(df)
KMeans_results = model.labels_

In [ ]:
len(KMeans_results)

#### DBSCAN clusterring

In [ ]:
### eps = the min distance for the clustering internals 
### min samples - min samples for a cluster 
### all others will consider outliers and will reduced from the model
eps_values =np.arange(2,4,1)
min_samples =  list(range(9,11,1))

In [ ]:
print(eps_values)
min_samples

In [ ]:
best_score = -1
best_eps = 0
best_min_samples = 0
for i in eps_values:
    for j in min_samples:
        model = DBSCAN(eps=i, min_samples=j).fit(df_std_num)
        model.labels_
        print(i, model.labels_)
        silhouette_avg = silhouette_score(df_std_num,model.labels_)
        print("For eps = ", i,'For min_val= ',j ,"The average silhouette_score is :", silhouette_avg)
        if silhouette_avg > best_score:
            best_score =  silhouette_avg
            best_eps = i
            best_min_samples = j

In [ ]:
silhouette_avg = silhouette_score(df_std_num,model.labels_)

In [ ]:
silhouette_avg

In [ ]:
len(model.labels_)

In [ ]:
set(model.labels_)

In [ ]:
list(model.labels_).count(-1)

In [ ]:
print("Best eps =", best_eps," | Best min_sample :", best_min_samples,'No. of clusters: ', len(set(model.labels_))," | Best silhouette_score is :", best_score)
best_eps

In [ ]:
model = DBSCAN(eps=best_eps, min_samples=best_min_samples).fit(df_std_num)
DBSCAN_results = model.labels_

In [ ]:
len(set(DBSCAN_results))

#### MeanShift clustering


In [ ]:
from sklearn.cluster import estimate_bandwidth
estimate_bandwidth(df)

In [ ]:
model = MeanShift(bandwidth=estimate_bandwidth(df)).fit(df)
model.labels_

In [ ]:
silhouette_score = silhouette_score(df, model.labels_)
silhouette_score

In [ ]:
print('silhouette_score: ' , silhouette_score, ' | No. of clusters: ', len(set(model.labels_)))
len(set(model.labels_))

In [ ]:
from sklearn.cluster import MeanShift
from sklearn.cluster import estimate_bandwidth
model = MeanShift(bandwidth=estimate_bandwidth(df_std_num)).fit(df_std_num)

In [ ]:
from sklearn.metrics import silhouette_score
silhouette_score = silhouette_score(df_std_num, model.labels_)
silhouette_score

In [ ]:
MeanShift_results = model.labels_

In [ ]:
print('silhouette_score: ' , silhouette_score, ' | No. of clusters: ', len(set(model.labels_)))

In [ ]:
set(model.labels_)

#### Birch clustering

In [ ]:
n_clusters =np.arange(3,10,1)
n_clusters

In [ ]:
from sklearn.metrics import silhouette_score
from sklearn.cluster import Birch
best_score = -1
best_no_clusters = 0
for i in n_clusters:
    model = Birch(n_clusters= i, threshold= 0.3).fit(df_std_num)
    print(i, model.labels_)
    silhouette_avg = silhouette_score(df_std_num,model.labels_)
    print("For n_clusters =", model.n_clusters,"The average silhouette_score is :", silhouette_avg)
    if silhouette_avg > best_score:
        best_score =  silhouette_avg
        best_no_clusters = model.n_clusters

In [ ]:
set(model.labels_)


In [ ]:
silhouette_score(df_std,model.labels_)

In [ ]:
print('Best No. of clusters: ', best_no_clusters," | Best silhouette_score is :", best_score)

In [ ]:
model = Birch(n_clusters= best_no_clusters, threshold= 0.3).fit(df_std_num)
Birch_results = model.labels_
len(Birch_results)

In [ ]:
set(model.labels_)

In [ ]:
#export base table
df.to_csv('Car_processed.csv', index= False)

In [ ]:
df_cl = df.copy()

In [ ]:
df_cl['Birch'] = Birch_results
#df_cl['MeanShift'] = MeanShift_results
# df_cl['DBSCAN'] = DBSCAN_results
#df_cl['KMeans'] = KMeans_results

df_cl

In [ ]:
df_cl.pivot_table(columns= 'owner', index= 'Birch', values = 'selling_price', aggfunc='mean')

In [ ]:
#export table with Clusters assigned
df_cl.to_csv('Car_processed_cl.csv', index= False)

# Exit Script